<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2020/Chapter_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 20 Basic file wrangling

# 20.2 Scenario: The product feed from hell

### Quick Check : Consider the choices

What are your options for handling the tasks I've identified? What modules in the standard library can you think of that will do the job? If you want, you can even stop right now and work out the code to do it. Then compare your solution with the one you develop later.

#### Discussion

From the standard library, use datetime for managing the dates/times of the files, and either os.path and os or pathlib for renaming and archiving the files.


In [2]:
# run this cell first to create files

! touch item_info.txt item_attributes.txt related_items.txt

In [3]:
import pathlib
cur_path = pathlib.Path(".")
FILE_PATTERN = "*.txt"
path_list = cur_path.glob(FILE_PATTERN)
print(list(path_list))

[PosixPath('related_items.txt'), PosixPath('item_info.txt'), PosixPath('item_attributes.txt')]


### Listing 20.1 File files_01.py


In [4]:
# Listing 20.1 File files_01.py

import datetime
import pathlib

FILE_PATTERN = "*.txt"             #A
ARCHIVE = "archive"

def main():

    date_string = datetime.date.today().strftime("%Y-%m-%d")    #B

    cur_path = pathlib.Path(".")
    archive_path = cur_path.joinpath(ARCHIVE)
    archive_path.mkdir(exist_ok=True)        #C

    paths = cur_path.glob(FILE_PATTERN)

    for path in paths:
        new_filename = f"{path.stem}_{date_string}{path.suffix}"
        new_path = archive_path.joinpath(new_filename)        #D
        path.rename(new_path)                      #E

if __name__ == '__main__':
     main()

### Quick Check: Potential Problems
Because the preceding solution is very simple, there are likely to be many situations that it won’t handle well. What are some potential issues or problems that might arise with the example script? How might you remedy these problems?

Consider the naming convention used for the files, which is based on the year, month and day, in that order. What advantages do you see in that convention? What might be the disadvantages? Can you make any arguments for putting the date string somewhere else in the filename, such as the beginning or the end?

#### Discussion

Multiple files during the same day would be a problem, for one thing. If you have lots of files, navigating the archive directory will become increasingly  difficult.

Using year-month-day date formats makes a text-based sort of the files sort by date as well. Putting the date at the end of the filename but before the extension makes it more difficult to parse the date element visually.

# 20.3 More organization

### Listing 20.2 File files_02.py

In [5]:
# run this before running cell below
# clear files from archive directory

! rm archive/*
! touch item_info.txt item_attributes.txt related_items.txt

In [6]:
# Listing 20.2 File files_02.py

import datetime
import pathlib

FILE_PATTERN = "*.txt"
ARCHIVE = "archive"

def main():

    date_string = datetime.date.today().strftime("%Y-%m-%d")

    cur_path = pathlib.Path(".")

    archive_path = cur_path.joinpath(ARCHIVE)
    archive_path.mkdir(exist_ok=True)                             #A
    new_path = archive_path.joinpath(date_string)
    new_path.mkdir(exist_ok=True)            #B

    paths = cur_path.glob(FILE_PATTERN)

    for path in paths:
        path.rename(new_path.joinpath(path.name))

if __name__ == '__main__':
     main()

### Try This: Implementation of multiple directories

How would you modify the code that you developed to archive each set of files in subdirectories named according to date received? Feel free to take the time to implement the code and test it.

### Quick Check: Alternate solutions
How might you create a script that does the same thing without using pathlib? What libraries and functions would you use?

#### Discussion
You'd use the os.path and os libraries—specifically, `os.path.join()`, `os.mkdir()`, and `os.rename()`.

In [ ]:
# @title
import datetime
import pathlib

FILE_PATTERN = "*.txt"
ARCHIVE = "archive"

if __name__ == '__main__':

    date_string = datetime.date.today().strftime("%Y-%m-%d")

    cur_path = pathlib.Path(".")

    new_path = cur_path.joinpath(ARCHIVE, date_string)
    new_path.mkdir()

    paths = cur_path.glob(FILE_PATTERN)

    for path in paths:
        path.rename(new_path.joinpath(path.name))

## 20.4.1 Compressing files

### Listing 20.3 File files_03.py

### Try This: Archiving to zip files pseudocode

Write the pseudocode for a solution that stores data files in zip files. What modules and functions or methods do you intend to use? Try coding your solution to make sure that it works.

#### Discussion
Pseudocode:
```
create path for zip file
create empty zipfile
for each file
    write into zipfile
    remove original file
```
(See the next section for sample code that does this.)

In [7]:
# run this before running cell below
# clear files from archive directory

! rm -rfr archive/*
! touch item_info.txt item_attributes.txt related_items.txt

In [8]:
# Listing 20.3 File files_03.py

import datetime
import pathlib
import zipfile          #A

FILE_PATTERN = "*.txt"
ARCHIVE = "archive"

def main():

    date_string = datetime.date.today().strftime("%Y-%m-%d")

    cur_path = pathlib.Path(".")
    archive_path = cur_path.joinpath(ARCHIVE)
    archive_path.mkdir(exist_ok=True)

    paths = cur_path.glob(FILE_PATTERN)

    zip_file_path = cur_path.joinpath(ARCHIVE, date_string + ".zip")   #B
    zip_file = zipfile.ZipFile(str(zip_file_path), "w")       #C

    for path in paths:
        zip_file.write(str(path))                                 #D
        path.unlink()             #E

if __name__ == '__main__':
     main()

## 20.4.2 Grooming files

### Listing 20.4 File files_04.py

In [9]:
# run this before running cell below
# create zip files in archive directory
from datetime import datetime, timedelta


def populate_archive(zip_file_path, current_date):
    for days in range(30, 40):
        zip_date = current_date - timedelta(days=days)
        new_zip_path = zip_file_path.joinpath(f"{zip_date.strftime('%Y-%m-%d')}.zip")
        zip_file = new_zip_path.write_text("Test")

cur_path = pathlib.Path(".")
zip_file_path = cur_path.joinpath(ARCHIVE)
current_date = datetime.today()
populate_archive(zip_file_path, current_date)

In [10]:
# Listing 20.4 File files_04.py

from datetime import datetime, timedelta
import pathlib
import zipfile

FILE_PATTERN = "*.zip"
ARCHIVE = "archive"
ARCHIVE_WEEKDAY = 1
def main():
    cur_path = pathlib.Path(".")
    zip_file_path = cur_path.joinpath(ARCHIVE)

    paths = zip_file_path.glob(FILE_PATTERN)
    current_date = datetime.today()    #A

    for path in paths:
        name = path.stem              #B
        path_date = datetime.strptime(name, "%Y-%m-%d")     #C
        path_timedelta = current_date - path_date          #D
        if (path_timedelta > timedelta(days=30)
                and path_date.weekday() != ARCHIVE_WEEKDAY):    #E
            path.unlink()

if __name__ == '__main__':
     main()

### Quick Check: Consider different parameters

Take some time to consider different grooming options. How would you modify the code in the previous Try This to keep only one file a month? How would you change the code so that files from the previous month and older are groomed to save one a week? (Note: This is not the same as older than 30 days!)

#### Discussion
You could use something similar to the code above but also check the month of the file against the current month.